#Phase 2: Modeling & Hyperparameter Tuning 

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt #graphs 
import tensorflow as tf 

**Train, Test, Validation**

In [2]:
#Split validation set by 20%
#Image generator helps creating image aumentation to increase the amount of data we have
#Will implement various rotations and flips to the images to distort
train_gen = ImageDataGenerator(rescale = 1./255, validation_split=0.20)
test_gen = ImageDataGenerator(rescale = 1./255)


def get_train_set(train_gen, input_size):
  train_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                         class_mode='categorical',
                                         target_size= input_size,
                                         color_mode = 'grayscale',
                                         batch_size= 32, 
                                         shuffle = True, 
                                         subset ='training')
  return train_set

def get_validation_set(train_gen, input_size):
  validation_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                          target_size= input_size,
                                          color_mode = 'grayscale',
                                          class_mode='categorical',
                                          batch_size= 32, 
                                          shuffle = True, 
                                          subset ='validation')
  return validation_set

def get_test_set(test_gen, input_size):
  test_set = test_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/test",
                                         target_size=input_size, 
                                         color_mode = 'grayscale',
                                         class_mode='categorical',
                                         batch_size= 1,
                                         shuffle = True)
  return test_set

In [3]:
#get train, val, test sets 
train_set = get_train_set(train_gen=train_gen, input_size = (224,224))
val_set = get_validation_set(train_gen=train_gen, input_size = (224,224))
test_set = get_test_set(test_gen=test_gen, input_size = (224,224))

Found 6755 images belonging to 196 classes.
Found 1585 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


#Modeling & Structural Tuning

In [4]:
!pip install -q -U keras-tuner

In [5]:
from tensorflow.keras.applications import xception #most parameters (44 mil)
import keras_tuner as kt
from kerastuner.tuners import Hyperband
from kerastuner import HyperModel

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#input, conv2d, pooling,flatten ,Dense, output
#flatten layer = get vector to put in classifier 
# conv layers followed by max pool
# last conv layer followed by a dropout layer to prevent overfitting 
# flatten, fully connected layers 

#TODO: make the number of conv2d layers dynamic

class Hypermodel(HyperModel):

  def __init__(self, num_Conv2D):
    self.num_Conv2D = num_Conv2D

  def build(self, hp):
    if (self.num_Conv2D <= 0):
      return -1

    model = tf.keras.Sequential()
    
    if (self.num_Conv2D == 1):
      model.add(tf.keras.layers.Conv2D(filters = hp.Choice('num_filters', values = [32,64], default = 64), kernel_size=(3,3),activation = 'relu',input_shape = (224,224,1)))
      model.add(tf.keras.layers.Dropout(hp.Float('dropout',
                                                  min_value=0.0,
                                                  max_value=0.1,
                                                  default=0.005,
                                                  step=0.01)))
      model.add(tf.keras.layers.MaxPool2D(3,3))
    else: 
      temp = self.num_Conv2D
      while (temp-2 > 0):
        model.add(tf.keras.layers.Conv2D(filters = hp.Choice('num_filters', values  = [32,64], default = 64), kernel_size=(3,3),activation='relu'))
        model.add(tf.keras.layers.MaxPool2D(3,3))
        temp = temp -1

      model.add(tf.keras.layers.Conv2D(filters = hp.Choice('num_filters', values  = [32,64], default = 64), kernel_size=(3,3),activation='relu'))
      model.add(tf.keras.layers.Dropout(hp.Float('dropout',min_value=0.0,max_value=0.1,default=0.005,step=0.01)))
      model.add(tf.keras.layers.MaxPool2D(3,3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value = 32, max_value=512,step=32),activation= 'relu'))
    tf.keras.layers.Dense(196, activation='softmax')
    model.compile(optimizer= tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3])),loss='categorical_crossentropy',metrics=['accuracy'])

    return model 

In [7]:
#don't explore unlikely options 
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
hypermodel_1 = Hypermodel(num_Conv2D=1)
hypermodel_2 = Hypermodel(num_Conv2D=2)
hypermodel_3 = Hypermodel(num_Conv2D=3)


tuner_1 = Hyperband(hypermodel=hypermodel_1, objective='val_accuracy', max_epochs= 100)
tuner_2 = Hyperband(hypermodel=hypermodel_2, objective='val_accuracy', max_epochs= 100)
tuner_3 = Hyperband(hypermodel =hypermodel_3, objective = 'val_accuracy', max_epochs = 100)

## 1 Convolutional Layer

In [8]:
tuner_1.search_space_summary()

Search space summary
Default search space size: 4
num_filters (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout (Float)
{'default': 0.005, 'conditions': [], 'min_value': 0.0, 'max_value': 0.1, 'step': 0.01, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001], 'ordered': True}


In [ ]:
tuner_1.search(train_set, epochs = 50, validation_data = val_set, callbacks = [stop_early])


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_filters       |64                |?                 
dropout           |0.03              |?                 
units             |448               |?                 
learning_rate     |0.001             |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |4                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2


## 2 Convolutional Layers

In [ ]:
tuner_2.search_space_summary()

In [ ]:
tuner_2.search(train_set, epochs = 50, validation_data = val_set, callbacks = [stop_early])

## 3 Convolutional Layers

In [ ]:
tuner_3.search_space_summary()

In [ ]:
tuner_3.search(train_set, epochs = 50, validation_data = val_set, callbacks = [stop_early])

**Epoch Optimization**

In [ ]:
# get best hp
best_hps = tuner_1.get_best_hyperparameters(num_trials = 1)[0]
#build best model 
model = tuner_1.hypermodel.build(best_hps)
hist = model.fit(train_set,epochs = 100, validation_data = val_set)

#get best epoch 
val_acc_per_epoch = hist.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+ 1

print('Best Epoch: %d' % (best_epoch))